# i.MX 93 encrypted and signed image

This notebook describes how to build the bootable AHAB image with ELE firmware and user application (led blinky in this example) and sign it and encrypt it using the secure EdgeLock enclave.

## Set up the environment

- SPSDK is needed with examples extension. pip install spsdk[examples] (refer to the installation documentation)
- This demo was tested with i.MX93 EVK board.


# Flashloader

We need to load flashloader first in order to communicate with the ELE using the mboot protocol. This is better described in "imx93_flashloader" notebook. Let's run it.



In [1]:
%run ./imx93_flashloader.ipynb

Created `%!` as an alias for `%execute`.
nxpimage -v ahab export ahab_config_flashloader_signed256.yaml 
INFO:spsdk.apps.nxpimage:Created AHAB Image:
Name:      AHAB Image
Starts:    0x0
Ends:      0x147af
Size:      83.9 kB
Alignment: 1 B
AHAB Image for mx93_a0

INFO:spsdk.apps.nxpimage:Created AHAB Image memory map:

+==0x0000_0000= AHAB Image =======================+
|                  Size: 83.9 kB                  |
|             AHAB Image for mx93_a0              |
|+==0x0000_0000= AHAB Container 0 ===============+|
||                  Size: 544 B                  ||
||        AHAB Container for nxp_SWver:0         ||
|+==0x0000_021f==================================+|
|                   Gap: 480 B                    |
|+==0x0000_0400= AHAB Container 1 ===============+|
||                  Size: 544 B                  ||
||        AHAB Container for oem_SWver:0         ||
|+==0x0000_061f==================================+|
|                   Gap: 6.6 kB                   |
|+=

INFO:libusbsio:Loading SIO library: C:\Users\nxf46245\spsdk\venv\Lib\site-packages\libusbsio\bin\x64\libusbsio.dll
INFO:libusbsio:HID enumeration[2294280356624]: initialized
INFO:libusbsio:HID enumeration[2294280356624]: finished, total 14 devices
INFO:libusbsio.hidapi.dev:Opening HID device at path: 'b'\\\\?\\hid#vid_1fc9&pid_014e#6&6466ab0&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}''
INFO:libusbsio.hidapi.dev:HID device 2294237082624 is now open
INFO:libusbsio.hidapi.dev:HID device 2294237082624 closed


nxpdevscan -p 
-------- Connected NXP UART Devices --------

Port: COM41
Type: mboot device

blhost -p com41 get-property 1 
Response status = 0 (0x0) Success.
Response word 1 = 1258424320 (0x4b020800)
Current Version = K2.8.0
blhost -p com41 get-property 18 
Response status = 0 (0x0) Success.
Response word 1 = 2442643059 (0x9197c673)
Response word 2 = 2573725851 (0x9967f09b)
Response word 3 = 1195651327 (0x474430ff)
Response word 4 = 1979819762 (0x7601a6f2)
Unique Device Identification = 73 C6 97 91 9B F0 67 99 FF 30 44 47 F2 A6 01 76
blhost -p com41 get-property 7 
Response status = 0 (0x0) Success.
Response word 1 = 25227228 (0x180efdc)
Available Commands = ['ReadMemory', 'WriteMemory', 'FillMemory', 'GetProperty', 'ReceiveSBFile', 'Execute', 'Call', 'Reset', 'SetProperty', 'FlashProgramOnce', 'FlashReadOnce', 'FlashReadResource', 'UpdateLifeCycle', 'EleMessage']
blhost -p com41 read-memory 0x304f0000 100 -h 
Reading memory
00000000: 5E DB D7 6E 7D BB 0D B6  AC 9F F3 E1 ED 53 E4 DD 

# NXPELE keyblob generation

We can create DEK keyblob for image encryption using the *nxpele* application.
For this demo we have chosen AES CBC mode with key size 128 bits. 

In [2]:
CONNECTION = "-p com41"

! nxpele $VERBOSITY $CONNECTION -f mx93 generate-keyblob DEK -a AES_CBC -i 0 -k inputs/keys/dek_key.txt -s 128 -o outputs/dek_keyblob.bin 

INFO:spsdk.ele.ele_comm:ELE communicator is using 1024 B size buffer at 304B0000 address in mx93 target.
INFO:spsdk.mboot.mcuboot:Connect: com41
INFO:spsdk.mboot.mcuboot:CMD: WriteMemory(address=0x304B0000, length=32, mem_id=0)
INFO:spsdk.mboot.mcuboot:CMD: GetProperty('MaxPacketSize', index=0)
INFO:spsdk.mboot.mcuboot:CMD: Max Packet Size = 512
INFO:spsdk.mboot.mcuboot:CMD: Successfully Send 32 out of 32 Bytes
INFO:spsdk.mboot.mcuboot:CMD: WriteMemory(address=0x304B0020, length=24, mem_id=0)
INFO:spsdk.mboot.mcuboot:CMD: GetProperty('MaxPacketSize', index=0)
INFO:spsdk.mboot.mcuboot:CMD: Max Packet Size = 512
INFO:spsdk.mboot.mcuboot:CMD: Successfully Send 24 out of 24 Bytes
INFO:spsdk.mboot.mcuboot:CMD: EleMessage Command (cmdMsgAddr=0x304B0000, cmdMsgCnt=8)
INFO:spsdk.mboot.mcuboot:CMD: EleMessage Response (respMsgAddr=0x304B0038, respMsgCnt=2)
INFO:spsdk.mboot.mcuboot:CMD: ReadMemory(address=0x304B0038, length=8, mem_id=0)
INFO:spsdk.mboot.mcuboot:CMD: Successfully Received 8 from 

## Image preparation
Now we should be ready to prepare the AHAB image for CM33 core containing ELE firmware and our application. We can use SPSDK *nxpimage* tool for image preparation.
Configuration file that generates signed and encrypted AHAB image containing ELE FW and flashloader might look like this:

```yaml
---
# ----------------------------------------------------------------------------------------------------
# ===========  Advanced High-Assurance Boot Configuration template for rt1180.  ===========
# ----------------------------------------------------------------------------------------------------
#                                        == General Options ==
family: mx93 # [Required], MCU family, Family identifier including the chip revision. If revision is not present, latest revision is used as default., Possible options:['rt118x']
revision: a0 # [Optional], MCU revision, Revision of silicon, Possible options:['a0']
image_type: serial_downloader # [Required], Type of image, The final use of image, this setting is changing the style of offsets in final container., Possible options:['xip', 'non_xip', 'serial_downloader']
output: outputs/led_blink.bin # [Required], Output AHAB file name, Revision of silicon
containers: # [Required], List of containers present in AHAB., The order of containers in the list defines the order in AHAB.
  - # ----------------------------------------------------------------------------------------------------
    #                     == Binary Container format to add to AHAB image ==
    # ----------------------------------------------------------------------------------------------------
    binary_container: # [Required], Binary AHAB container
      path: inputs/mx93a0-ele-fw.img # [Required], The AHAB container binary file, The binary file that contains AHAB "my_binary_container.bin
  - # [Example of possible configuration #1]
    # ----------------------------------------------------------------------------------------------------
    #                  == Configuration Container format to add to AHAB image ==
    # ----------------------------------------------------------------------------------------------------
    container: # [Required], AHAB Container
      srk_set: oem # [Required], Super Root Key (SRK) set, Defines which set is used to authenticate the container., Possible options:['none', 'oem', 'nxp']
      used_srk_id: 0 # [Conditionally required], Used SRK, Which key from SRK set is being used.
      srk_revoke_mask: 0 # [Optional], SRK revoke mask, Bitmask to indicate which SRKs to revoke. Bit set to 1 means revoke key. Bit 0 = revoke SRK_0, bit 1 = revoke SRK_1 etc.
      fuse_version: 0 # [Required], Fuse version, The value must be equal or greater than the version stored in fuses to allow loading this container.
      sw_version: 0 # [Required], Software version, Number used by Privileged Host Boot Companion (PHBC) to select between multiple images with same Fuse version field.
      signing_key:
        inputs/keys/ecc256/srk0_ecc256.pem # [Conditionally required], AHAB container signing key, Private key used for sign the container header. Header can be signed by SRK or by image key that was signed by SRK. If an image key is used, it must be the same algorithm and key size as the SRK. In both cases, the referenced SRK must not have been revoked.
        # ----------------------------------------------------------------------------------------------------
        #               == Configuration of AHAB Container images (array of multiple images) ==
        # ----------------------------------------------------------------------------------------------------
      images: # [Required], Image array, Array of image entries.
        - image_path: inputs/mx93_led_blink_0x304a0000.bin # [Required], Image path, Path to image binary (absolute/relative).
          image_offset: "0x10000" # [Required], Image offset in AHAB container, Relative address for start of AHAB image (can contain multiple AHAB containers). In case of XiP type of AHAB image, the load_address and entry_point must correspond to this values. Example of setting of load_address - AHAB_IMAGE_ADDRESS+IMAGE_OFFSET=LOAD_ADDRESS
          load_address: "0x304a0000" # [Required], Image destination address, Address the image is written to in memory (absolute address in system memory).
          entry_point: "0x304a0000" # [Required], Image entry point, Image entry point (absolute address). Valid only for executable image types.
          image_type: executable # [Required], Image type, Kind of image., Possible options:['executable', 'data', 'dcd_image', 'seco', 'provisioning_image', 'provisioning_data']
          core_id: cortex-m33 # [Required], Core ID, Defines the core the image is dedicated for., Possible options:['cortex-m33', 'cortex-m7']
          is_encrypted: true # [Required], Image encryption, Determines, whether image is encrypted or not.
          boot_flags: 0 # [Optional], Boot flags, Boot flags controlling SCFW boot.
          meta_data_start_cpu_id: 0 # [Optional], Start CPU ID, Resource ID of CPU to be started
          meta_data_mu_cpu_id: 0 # [Optional], CPU memory unit start ID, Resource ID of the MU associated with the CPU
          meta_data_start_partition_id: 0 # [Optional], Start partition ID, Partition ID of the partition to start
          hash_type: sha384 # [Optional], Images HASH type, HASH type of image. All images in the container must have the same HASH type., Possible options:['sha256', 'sha384', 'sha512']
        # ----------------------------------------------------------------------------------------------------
        #                                == Configuration of AHAB SRK table ==
        # ----------------------------------------------------------------------------------------------------
      srk_table: # [Conditionally required], SRK Table, SRK (Super Root key) table definition.
        srk_array: # [Required], Super Root Key (SRK) table, Table containing the used SRK records. All SRKs must be of the same type. Supported signing algorithms are: RSASSA-PSS or ECDSA. Supported hash algorithms: sha256, sha384, sha512. Supported key sizes/curves: prime256v1, sec384r1, sec512r1, rsa2048, rsa4096. Certificate may be of Certificate Authority.
          - inputs/keys/ecc256/srk0_ecc256.pub # SRK key, Path to SRK Key file.
          - inputs/keys/ecc256/srk1_ecc256.pub # SRK key, Path to SRK Key file.
          - inputs/keys/ecc256/srk2_ecc256.pub # SRK key, Path to SRK Key file.
          - inputs/keys/ecc256/srk3_ecc256.pub # SRK key, Path to SRK Key file.
      blob: # [Optional], Encryption blob, Encryption blob container definition
        key_identifier: 0
        dek_key_size: 128 # [Required], DEK key size, Data Encryption key size. Used for AES CBC-MAC (128/192/256 size), Possible options:[128, 192, 256]
        dek_key: inputs/keys/dek_key.txt # [Required], DEK key, Data Encryption key. Used for AES CBC-MAC (128/192/256 size). The HEX format is accepted
        dek_keyblob: outputs/dek_keyblob.bin # [Required], DEK keyblob, Wrapped Data Encryption key. Used for AES CBC-MAC (128/192/256 size). The HEX format is accepted

``` 

All necessary files are prepared in the *inputs* directory. 

In [6]:
%alias execute echo %l && %l
%alias_magic ! execute

WORKSPACE = "workspace/" # change this to path to your workspace
VERBOSITY = "-v" # verbosity of commands, might be -v or -vv for debug or blank for no additional info
AHAB_CONFIGURATION = "ahab_config_led_blink_encrypted_signed256.yaml"
AHAB_IMAGE = "outputs/led_blink.bin"

# export AHAB image with flashloader
%! nxpimage $VERBOSITY ahab export -c $AHAB_CONFIGURATION 



Created `%!` as an alias for `%execute`.
nxpimage -v ahab export -c ahab_config_led_blink_encrypted_signed256.yaml 
Success. (AHAB: C:/Users/nxf46245/spsdk/examples/jupyter_examples/imx93/outputs/led_blink.bin created.)
Generated SRK hash files (c:\Users\nxf46245\spsdk\examples\jupyter_examples\imx93\led_blink_oem1_srk_hash*.*).


INFO:spsdk.apps.nxpimage:Created AHAB Image:
Name:      AHAB Image
Starts:    0x0
Ends:      0x11b8f
Size:      Size: 72.6 kB; 72,592 B
Alignment: 1 B
AHAB Image for mx93_a0

INFO:spsdk.apps.nxpimage:Created AHAB Image memory map:

+==0x0000_0000= AHAB Image =======================+
|             Size: 72.6 kB; 72,592 B             |
|             AHAB Image for mx93_a0              |
|+==0x0000_0000= AHAB Container 0 ===============+|
||                  Size: 544 B                  ||
||        AHAB Container for nxp_SWver:0         ||
|+==0x0000_021f==================================+|
|                   Gap: 480 B                    |
|+==0x0000_0400= AHAB Container 1 ===============+|
||                  Size: 616 B                  ||
||        AHAB Container for oem_SWver:0         ||
|+==0x0000_0667==================================+|
|                   Gap: 6.6 kB                   |
|+==0x0000_2000= Container 0 AHAB Data Image 0 ==+|
||            Size: 58.1 kB; 58,144 B   

# Fuse write and lifecycle move

We need to write fuses and move the lifecycle to OEM_CLOSED in order to lock the part. Writing fuses can be done with the BCF (blhost batch file) script that was generated together with the AHAB image. Moving the lifecycle is done using the blhost *update-life-cycle*. The last step will be to reset the chip using the *nxpele* application.

The script for writing fuses with blhost is automatically generated.
```yaml
# BLHOST SRK Hash fuses programming script
# Generated by SPSDK 1.11.0
# Chip: mx93 rev:a0
# SRK Hash(Big Endian): cb2cc774b2dcec92c840eca0646b78f8d3661d3a43ed265a490a13aca75e190a

#  OEM SRKH7 fuses.
efuse-program-once 0x80 0x74c72ccb
#  OEM SRKH6 fuses.
efuse-program-once 0x81 0x92ecdcb2
#  OEM SRKH5 fuses.
efuse-program-once 0x82 0xa0ec40c8
#  OEM SRKH4 fuses.
efuse-program-once 0x83 0xf8786b64
#  OEM SRKH3 fuses.
efuse-program-once 0x84 0x3a1d66d3
#  OEM SRKH2 fuses.
efuse-program-once 0x85 0x5a26ed43
#  OEM SRKH1 fuses.
efuse-program-once 0x86 0xac130a49
#  OEM SRKH0 fuses.
efuse-program-once 0x87 0x0a195ea7
```

In [4]:
# !!! CAUTION the below commands lock the part !!! #
# write fuses
#! blhost $CONNECTION batch outputs/flashloader_signed_oem1_srk_hash_blhost.bcf

# move lifecycle
#! blhost $CONNECTION update-life-cycle 8

# Reset the chip
! nxpele -f mx93 $CONNECTION reset


ERROR:SPSDK: ELE Communication failed with mBoot: MBoot: EleMessage interrupted -> No response packet from target device


## Image download
We use SPSDK tool *sdpshost* to load the image. So at first, we need to set up board boot configuration to Cortex-M serial downloader mode (1011) and load the image to OCRAM. Application should start immediately after loading the image.

In [5]:
# use SDPSHOST tool to download the image
! sdpshost -u mx93 -n MX93 write-file $AHAB_IMAGE
# use UUU tool to download the image
# %! uuu -b emmc inputs/uboot_singleboot_flash.bin $AHAB_IMAGE 


!!! THIS IS AN EXPERIMENTAL UTILITY! USE WITH CAUTION !!!


!!! THIS IS AN EXPERIMENTAL UTILITY! USE WITH CAUTION !!!

